# Forced alignment with CLAP-IPA
This example demonstrates how to perform forced alignment in speech recordings.

## Load IPA-Aligner

IPA-Aligner has two components: 
1. The speech encoder for the recording you want to search. (E.g., `sample.wav`)
2. The phone encoder for the phone sequence you want to search for (E.g., [bo:do:])

In [ ]:
import torch
from clap.encoders import SpeechEncoder, PhoneEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

speech_encoder = SpeechEncoder.from_pretrained('anyspeech/ipa-align-base-speech')
phone_encoder = PhoneEncoder.from_pretrained('anyspeech/ipa-align-base-phone')

phone_encoder.eval().to(device)
speech_encoder.eval().to(device)

# Load example dataset


In [2]:
from datasets import load_dataset

ds = load_dataset("anyspeech/frame_labels")

In [3]:
#play audio
import numpy as np
from IPython.display import Audio
import librosa

audio = ds['train'][10]['audio']['array']
Audio(audio, rate=16000)

## Load processors 

In [4]:
from transformers import AutoProcessor, DebertaV2Tokenizer
# Encode the target phone sequence 
tokenizer = DebertaV2Tokenizer.from_pretrained('charsiu/IPATokenizer')
processor = AutoProcessor.from_pretrained('openai/whisper-base')

### Define some helper functions

In [5]:
import itertools
import numpy as np
from collections import defaultdict, Counter
from tqdm import tqdm 
from datasets import load_dataset
from lingpy.sequence.sound_classes import ipa2tokens
from librosa.sequence import dtw
import torch
import torch.nn.functional as F

def create_phone_mask(lengths):

    mask = torch.zeros(len(lengths),sum(lengths))

    cumsum = 0
    for i,l in enumerate(lengths):
        mask[i,cumsum:cumsum+l] = 1/l+1e-8
        cumsum += l

    return mask

def create_sliding_window(lengths,win_len=10,shift=5):

    L = torch.max(lengths)
    num_win = torch.ceil(L/shift).long()
    sliding_window = torch.zeros(num_win,L)
    for n in range(num_win):
        sliding_window[n,n*shift:min(n*shift+win_len,L)] = 1.0

    return sliding_window

def forced_align(cost,phone_feature):
    
    D,align = dtw(C=cost.T,
                  step_sizes_sigma=np.array([[1, 1], [0,1]]))
    #print(align)
    align_seq = [-1 for i in range(max(align[:,0])+1)]
    for i in list(align):
        
        if align_seq[i[0]]<i[1]:
            align_seq[i[0]]=i[1]
    align_id = list(align_seq)
    
    return [(i,p) for i,p in zip(align_id[:-1],phone_feature[1:-1])]

In [22]:
def IPA_Forced_Align(entry, speech_encoder, phone_encoder, tokenizer, processor,unit='phone',win_len=1,frame_shift=1):
    
    if unit == 'word':
        y = entry['word_detail']['start']
        y_ids = entry['word_detail']['utterance']
    elif unit == 'phone':
        y = entry['merge_phonetic_detail']['start']
        y_ids = entry['merge_phonetic_detail']['utterance']            
    
    
    input_features = [entry['audio']['array']]

    batch = processor(input_features, sampling_rate=16000, return_attention_mask=True,return_tensors="pt")
    
    length = torch.max(torch.sum(batch['attention_mask'],dim=-1))
    batch['attention_mask'] = batch['attention_mask'][:,:length]
    batch['input_features'] = batch['input_features'][:,:,:length]
    speech_lengths = torch.sum(batch['attention_mask'][:,::2],dim=-1)
    speech_sliding_mask = create_sliding_window(speech_lengths,win_len=win_len,shift=frame_shift)
    
    if unit == 'word':
        augmented_y_ids = ['[SEP]'] + y_ids + ['[SEP]']
    elif unit == 'phone':
        augmented_y_ids = y_ids
        y = y[1:-1]
        
    out = tokenizer(augmented_y_ids, return_attention_mask=False, return_length=True, return_token_type_ids=False, add_special_tokens=False)
    phones = torch.tensor(list(itertools.chain.from_iterable(out['input_ids']))).long().unsqueeze(0)
    phone_mask = create_phone_mask(out['length'])
    
    with torch.no_grad():
        speech_features = speech_encoder(**batch.to(device)).last_hidden_state.squeeze()

        phone_features = phone_encoder(phones.to(device)).last_hidden_state.squeeze()

    transformed_phone_features = torch.matmul(phone_mask.to(device),phone_features)
    transformed_speech_features = torch.matmul(speech_sliding_mask.to(device),speech_features)
    transformed_speech_features = F.normalize(transformed_speech_features,dim=-1)
    transformed_phone_features = F.normalize(transformed_phone_features,dim=-1)
    pairwise_cos_sim = torch.matmul(transformed_speech_features,transformed_phone_features.t())
    

    alignment = forced_align(-pairwise_cos_sim.cpu().numpy(),augmented_y_ids)
    #print(alignment)
    #alignment = list(torch.argmax(pairwise_cos_sim,dim=1).cpu().numpy())
    y_hat = [float(i*frame_shift*0.02) for i,p in alignment]
    y_hat_ids = [p for i,p in alignment]

    return y_hat, y_hat_ids, y

### Phone-level forced alignment

In [32]:
entry = ds['train'][10]
fa_time, phones, ground_truth = IPA_Forced_Align(entry, speech_encoder, phone_encoder, tokenizer, processor,unit='phone',win_len=1,frame_shift=1)

In [33]:
print(f"Phones:{phones}")
print(f"Forced Aligned Start Time:{fa_time}")
print(f"True Start Time:{ground_truth}")

Phones:['ʌ', 'm', 'i', 'b', 'ɪ', 'z', 't', 'tʃ', 'eɪ', 'n', 'd', 'ʃ', 'eɪ', 'p', 'k', 'ɑ', 'n', 's', 't', 'ɪ', 'n', 'l', 'i']
Forced Aligned Start Time:[0.24, 0.26, 0.36, 0.46, 0.54, 0.58, 0.64, 0.68, 0.8200000000000001, 0.96, 1.04, 1.12, 1.28, 1.42, 1.5, 1.62, 1.72, 1.78, 1.86, 1.92, 1.96, 2.02, 2.04]
True Start Time:[0.14, 0.225, 0.33, 0.47, 0.54125, 0.60375, 0.671875, 0.72375, 0.79625, 0.9869375, 1.0365, 1.0783125, 1.2349375, 1.4325, 1.54125, 1.621875, 1.76, 1.78125, 1.8525, 1.9125, 1.95, 1.985, 2.0575]


### Word-level alignment

In [34]:
entry = ds['train'][10]
fa_time, words, ground_truth = IPA_Forced_Align(entry, speech_encoder, phone_encoder, tokenizer, processor,unit='word',win_len=1,frame_shift=1)

In [35]:
print(f"Phones:{words}")
print(f"Forced Aligned Start Time:{fa_time}")
print(f"True Start Time:{ground_truth}")

Phones:['əˈmibəz', 'ˈtʃeɪndʒ', 'ˈʃeɪp', 'ˈkɑnstəntɫi']
Forced Aligned Start Time:[0.16, 0.6, 1.16, 1.5]
True Start Time:[0.14, 0.671875, 1.0783125, 1.54125]
